In [2]:
from model.BrownianBridge.LatentBrownianBridgeModel import LatentBrownianBridgeModel 
from model.VQGAN.vqgan import VQModel 
import torchvision.transforms as transforms
from PIL import Image
# from model.reg.reg import Reg, Transformer_2D

import torch 
# checkpoint = '/mnt/disk3/tiennh/taming-transformers/logs/2023-09-14T22-05-12_custom_vqgan/checkpoints/last.ckpt' 

/root/anaconda3/envs/BBDM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import matplotlib.pyplot as plt
from runners.base.EMA import EMA
# w_extract_features.ipynb

In [4]:
import yaml
import argparse 
import omegaconf.dictconfig
import os
f = open('configs/Template_CPDM.yaml', 'r')
dict_config = yaml.load(f, Loader=yaml.FullLoader)

IMAGE_SIZE = (256,256)
MAX_PIXEL = 2047

transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.0),
            transforms.Resize(IMAGE_SIZE),
            transforms.ToTensor()
        ])

def dict2namespace(config):
    namespace = argparse.Namespace()
    for key, value in config.items():
        if isinstance(value, dict) or isinstance(value, omegaconf.dictconfig.DictConfig):
            new_value = dict2namespace(value)
        else:
            new_value = value
        setattr(namespace, key, new_value)
    return namespace

nconfig = dict2namespace(dict_config)
nconfig.model.model_load_path = 'results1/229_conditional_LBBDM_segment_atten_2encoders/LBBDM-f4/checkpoint/top_model_epoch_48.pth'
nconfig.training.device = 'cuda:2'

vq = VQModel(**vars(nconfig.model.VQGAN.params))
model = LatentBrownianBridgeModel(nconfig.model)


# model.vqgan = vq.eval()



Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
Restored from /home/PET-CT/vqgan/vq1_3_69.ckpt
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
Restored from /home/PET-CT/vqgan/vq1_3_69.ckpt
load vqgan from /home/PET-CT/vqgan/vq1_3_69.ckpt
Spatial Rescaler mapping from 1 to 3 channels after resizing.
Spatial Rescaler mapping from 1 to 3 channels after resizing.


In [5]:
def load_model_from_checkpoint(config, net):
    model_states = None
    # if config.model.__contains__('model_load_path') and config.model.model_load_path is not None:
    print(f"load model {config.model.model_name} from {config.model.model_load_path}")
    model_states = torch.load(config.model.model_load_path, map_location='cpu')

    global_epoch = model_states['epoch']
    global_step = model_states['step']

    # load model
    
    ema = EMA(config.model.EMA.ema_decay)
    update_ema_interval = config.model.EMA.update_ema_interval
    start_ema_step = config.model.EMA.start_ema_step
    ema.register(net)

    net.load_state_dict(model_states['model'])
    
   
    # Load EMA
    if config.model.EMA.use_ema:
        ema.shadow = model_states['ema']
        ema.reset_device(net)
        ema.apply_shadow(net)
        
    net = net.to(config.training.device)
    return net, ema 

model, ema = load_model_from_checkpoint(nconfig, model)

load model LBBDM-f4 from results1/229_conditional_LBBDM_segment_atten_2encoders/LBBDM-f4/checkpoint/top_model_epoch_48.pth


In [6]:
unet_model = model.denoise_fn
# unet_model

In [7]:
def sample_to_eval( net, y, y_name, config, callback_save_features=None):
    to_normal = config.data.dataset_config.to_normal

    with torch.no_grad():
        y = y.to(config.training.device)
        net = net.to(config.training.device)
        out_samples, one_step_samples, _ = net.sample(y, [y_name], 'val_step', clip_denoised=False ,sample_mid_step=True, callback=callback_save_features)
            
        return out_samples, one_step_samples 

In [8]:
import torch 
import numpy as np  

index = 14211
x_np = np.load(f'/home/PET-CT/splited_data_15k/test/A/{index}.npy', allow_pickle=True)
x_np = x_np / float(MAX_PIXEL)
image = Image.fromarray(x_np)
x = transform(image)
x = x.unsqueeze(0)

In [9]:
root_dir_for_save_features = 'features/CPDM'
feature_maps_path = os.path.join(root_dir_for_save_features, str(index))
os.makedirs(feature_maps_path, exist_ok=True)

def save_feature_map(feature_map, dirname , filename):
        save_path = os.path.join(dirname, f"{filename}.pt")
        torch.save(feature_map, save_path)


def callback_save_features(img, pred_x0, i): 
    dir_name = os.path.join(feature_maps_path, 'input_blocks')
    os.makedirs(dir_name, exist_ok=True)
    save_feature_maps(unet_model.input_blocks, dir_name, i)
    dir_name = os.path.join(feature_maps_path, 'output_blocks')
    os.makedirs(dir_name, exist_ok=True)
    save_feature_maps(unet_model.output_blocks, dir_name, i)

def save_feature_maps(blocks, dir_name, i):
    block_idx = 0
    for block in blocks:
        if block_idx < 3:
            block_idx += 1
            continue
        if "ResBlock" in str(type(block[0])):
            in_layers_features = block[0].in_layers_features
            out_layers_features = block[0].out_layers_features
            if in_layers_features is not None:
                save_feature_map(in_layers_features, dir_name, f"block_{block_idx}_in_layers_features_time_{i}")
            else : 
                print(f"block {block_idx} timestep {i} has in_layers_features: None ")
                
            if out_layers_features is not None:
                save_feature_map(out_layers_features, dir_name, f"block_{block_idx}_out_layers_features_time_{i}")
            else :
                print(f"block {block_idx} timestep {i} has out_layers_features: None ")
        block_idx += 1

In [10]:
print(x.shape)
model.eval()

out_samples, one_step_samples = sample_to_eval(model, x, index, nconfig, callback_save_features)

torch.Size([1, 1, 256, 256])


save one step sample mid steps: 100%|██████████| 200/200 [00:05<00:00, 38.85it/s]


In [11]:
len(one_step_samples)

200

In [12]:
unet_model.output_blocks[4][0].in_layers_features.shape

torch.Size([1, 384, 16, 16])

In [13]:
len(unet_model.output_blocks)

12

In [14]:
midsteps = torch.arange(1000 - 1, 1, step=-((1000 - 1) / (200 - 2))).long()
steps = torch.cat((midsteps, torch.Tensor([1, 0]).long()), dim=0)

In [15]:
midsteps

tensor([999, 993, 988, 983, 978, 973, 968, 963, 958, 953, 948, 943, 938, 933,
        928, 923, 918, 913, 908, 903, 898, 893, 888, 882, 877, 872, 867, 862,
        857, 852, 847, 842, 837, 832, 827, 822, 817, 812, 807, 802, 797, 792,
        787, 782, 777, 771, 766, 761, 756, 751, 746, 741, 736, 731, 726, 721,
        716, 711, 706, 701, 696, 691, 686, 681, 676, 671, 666, 660, 655, 650,
        645, 640, 635, 630, 625, 620, 615, 610, 605, 600, 595, 590, 585, 580,
        575, 570, 565, 560, 555, 549, 544, 539, 534, 529, 524, 519, 514, 509,
        504, 499, 494, 489, 484, 479, 474, 469, 464, 459, 454, 449, 444, 438,
        433, 428, 423, 418, 413, 408, 403, 398, 393, 388, 383, 378, 373, 368,
        363, 358, 353, 348, 343, 338, 333, 327, 322, 317, 312, 307, 302, 297,
        292, 287, 282, 277, 272, 267, 262, 257, 252, 247, 242, 237, 232, 227,
        222, 216, 211, 206, 201, 196, 191, 186, 181, 176, 171, 166, 161, 156,
        151, 146, 141, 136, 131, 126, 121, 116, 111, 105, 100,  

In [16]:
steps

tensor([999, 993, 988, 983, 978, 973, 968, 963, 958, 953, 948, 943, 938, 933,
        928, 923, 918, 913, 908, 903, 898, 893, 888, 882, 877, 872, 867, 862,
        857, 852, 847, 842, 837, 832, 827, 822, 817, 812, 807, 802, 797, 792,
        787, 782, 777, 771, 766, 761, 756, 751, 746, 741, 736, 731, 726, 721,
        716, 711, 706, 701, 696, 691, 686, 681, 676, 671, 666, 660, 655, 650,
        645, 640, 635, 630, 625, 620, 615, 610, 605, 600, 595, 590, 585, 580,
        575, 570, 565, 560, 555, 549, 544, 539, 534, 529, 524, 519, 514, 509,
        504, 499, 494, 489, 484, 479, 474, 469, 464, 459, 454, 449, 444, 438,
        433, 428, 423, 418, 413, 408, 403, 398, 393, 388, 383, 378, 373, 368,
        363, 358, 353, 348, 343, 338, 333, 327, 322, 317, 312, 307, 302, 297,
        292, 287, 282, 277, 272, 267, 262, 257, 252, 247, 242, 237, 232, 227,
        222, 216, 211, 206, 201, 196, 191, 186, 181, 176, 171, 166, 161, 156,
        151, 146, 141, 136, 131, 126, 121, 116, 111, 105, 100,  

In [17]:
int = '10' 
int1 = int(int)

TypeError: 'str' object is not callable